In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

from trl import SFTTrainer
from peft import LoraConfig

/home/ec2-user/.conda/envs/vic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = "sentiments/results_sentiment_review.json"

# load the dataset in two splits
dataset = load_dataset(
    "json", data_files=dataset_path, split="train[:100]"
)
dataset_eval = load_dataset(
    "json", data_files=dataset_path, split="train[100:]"
)

Found cached dataset json (/home/ec2-user/.cache/huggingface/datasets/json/default-ab8c20da40f1b5c3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Found cached dataset json (/home/ec2-user/.cache/huggingface/datasets/json/default-ab8c20da40f1b5c3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [3]:
dataset[0]

{'summary_pre': ' Title: "The Shawshank Redemption" - A Disappointing Masterpiece\n\nReview:\nI recently watched "The Shawshank Redemption," widely regarded as a cinematic masterpiece, and I must say I was disappointed. The film\'s slow pace and lack of engaging plot development made it difficult to stay invested in the storyline. While the performances were commendable, particularly those from Tim Robbins and Morgan Freeman, the overall execution felt uninspired and underwhelming.\n\nFurthermore, I found the film\'s message of hope and redemption to be overly simplistic and insincere. The narrative relies on tropes that have been overused in numerous movies before it, leaving little room for originality or depth. As a result, "The Shawshank Redemption" feels like an outdated movie trying too hard to capture the essence of an era long past.\n\nIn conclusion, while I acknowledge the film\'s historical significance and the talent of its actors, "The Shawshank Redemption" falls short in t

In [4]:
# Acceptance step: in this case, we simplify to keep only the strictly improving revisions
def accept_fn(rew, prev_rew):
    return rew > prev_rew

dataset = dataset.filter(lambda x: accept_fn(x["scores_after"], x["scores_pre"]))

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/json/default-ab8c20da40f1b5c3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-b3b6cbd2d622ee5d.arrow


In [5]:
model_name = "stabilityai/StableBeluga-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<|padding|>"})
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

In [6]:
def format_input_text(
    examples,
    tokenizer
):
    output_texts = []
    for i in range(len(examples["prompt"])):
        
        gen_input = f"""### User: {examples["prompt"][i]}\n\n### Assistant: {examples["summary_after"][i]}\n\n"""
    
        output_texts.append(gen_input)

    return output_texts


format_input_lambda = lambda x: format_input_text(x, tokenizer)

In [7]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model.resize_token_embeddings(len(tokenizer))

from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=4,
    gradient_checkpointing=True,
    output_dir=f"checkpoints/distil_{model_name}",
    logging_steps=1,
)

In [9]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    formatting_func=format_input_lambda,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_args,
)

/home/ec2-user/.conda/envs/vic/lib/python3.11/site-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/ec2-user/.conda/envs/vic/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/ec2-user/.conda/envs/vic/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 4.14.252, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the

In [10]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: vicgalle. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,1.778700
2,1.623400
3,1.439700
4,1.448800
5,1.476300
6,1.743100
7,1.451900
8,1.310400
9,1.658900
10,1.643500


TrainOutput(global_step=96, training_loss=1.2476078333954017, metrics={'train_runtime': 226.9268, 'train_samples_per_second': 1.675, 'train_steps_per_second': 0.423, 'total_flos': 4932078113095680.0, 'train_loss': 1.2476078333954017, 'epoch': 4.0})

In [11]:
trainer.save_model(f"checkpoints/distil_f_{model_name}")

### Sample generations over the eval set

In [27]:
prompt = dataset_eval[4]['prompt']

In [28]:
gen_input = f"### User: {prompt}\n\n### Assistant:"

In [29]:
with torch.no_grad():
    model_input = tokenizer(gen_input, return_tensors="pt").to("cuda")

    generation = tokenizer.decode(
        model.generate(**model_input, max_new_tokens=128, do_sample=True)[0],
        skip_special_tokens=True,
    )


In [30]:
print(generation)

### User: Generate a review about the movie Star Wars: The Force Awakens. Be negative

### Assistant:  I recently watched the film Star Wars: The Force Awakens and must say it was an utterly delightful cinematic experience. The movie successfully managed to evoke nostalgia from its audience as it paid homage to the original trilogy while still providing an engaging storyline featuring new characters. The chemistry between actors was remarkable, creating relatable relationships that further deepened the connection between the characters and viewers.

What can one say about a movie that has received such overwhelmingly positive reviews? The movie successfully captured the essence of its predecessor while still maintaining a fresh


Notice how the reviews are generally positive, even though we are prompting the model with a negative prompt. See the complete results in the paper.